# 📘 BaseLine 預測誤差分佈圖 (Residual Histogram)

這段程式碼會要求使用者輸入股票代號（如 `3105`），讀取對應的 `.parquet` 檔案，
並將資料中所有欄位的數值分布以**兩張圖一排**的方式畫出直方圖。

### 📊 如何解讀 Residual Distribution（預測誤差分布圖）

殘差分布圖（Residual Distribution）是用來觀察模型預測誤差的圖表。以下是看圖時可以注意的幾個關鍵點：

#### 1. **分布形狀（Shape）**
- 理想情況下，分布應該是**對稱鐘型（近似常態分布）**，表示預測誤差平均為 0，沒有系統性偏誤。
- 若圖偏向一側，可能代表模型**有系統性高估或低估**。

#### 2. **集中程度（Spread / Variance）**
- 分布越窄、越尖，代表模型預測越穩定，誤差範圍小。
- 分布越寬、尾巴越長，表示模型誤差波動大，可能有較多 outlier。

#### 3. **平均位置（Centering around 0）**
- X 軸代表 residual 值（通常是 `True - Predicted`）。
- 若直方圖的中心落在 0 附近，代表預測**沒有偏誤**（Unbiased）。
- 若偏左（負值多）：模型偏高估；若偏右：模型偏低估。

#### 4. **尾端與異常值（Outliers）**
- 殘差分布的兩端（尾巴）可以看出模型是否容易出現大誤差。
- 長尾代表模型對某些狀況無法良好預測，可能要檢查資料異常或模型穩定性。

#### 5. **紅色虛線（基準線）**
- 圖中紅線通常標示 residual = 0 的位置，用來幫助視覺對齊中心點。

---

### 📌 小結

殘差圖是一種直觀檢查模型預測品質的工具，可用來判斷：
- 是否有偏誤？
- 預測是否穩定？
- 是否存在極端誤差？

搭配數值指標（如 RMSE、MAE）可以更全面地評估模型表現。


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

sys.path.append(os.path.abspath("../.."))
from config import BASELINE_RESULT_DIR

stock_code = input("Please enter stock id: ").strip()
filename   = f"{stock_code}.parquet"
input_path = os.path.join(BASELINE_RESULT_DIR, filename)
df = pd.read_parquet(input_path)

column_list = list(df.columns)

for i in range(0, len(column_list), 2):
    fig, axs = plt.subplots(1, 2, figsize=(14, 5))  # 1 row, 2 columns

    for j in range(2):
        if i + j < len(column_list):
            col = column_list[i + j]
            data = df[col].dropna()

            axs[j].hist(data, bins=50, edgecolor='black', alpha=0.7)
            axs[j].set_title(f'Residual Distribution: {col}')
            axs[j].set_xlabel('Prediction Error (Residual = True - Predicted)')
            axs[j].set_ylabel('Frequency')
            axs[j].grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
            axs[j].axvline(0, color='red', linestyle='--', linewidth=1)

    plt.tight_layout()
    plt.show()